In [213]:
import numpy as np
import pandas as pd

data = pd.read_csv('RealEstate.csv')

In [214]:
print data.head()

      MLS            Location     Price  Bedrooms  Bathrooms  Size  \
0  132842       Arroyo Grande  795000.0         3          3  2371   
1  134364         Paso Robles  399000.0         4          3  2818   
2  135141         Paso Robles  545000.0         4          3  3032   
3  135712           Morro Bay  909000.0         4          4  3540   
4  136282  Santa Maria-Orcutt  109900.0         3          1  1249   

   Price/SQ.Ft      Status  
0       335.30  Short Sale  
1       141.59  Short Sale  
2       179.75  Short Sale  
3       256.78  Short Sale  
4        87.99  Short Sale  


In [215]:
data.apply(lambda x: x.count(), axis=0)


MLS            781
Location       781
Price          781
Bedrooms       781
Bathrooms      781
Size           781
Price/SQ.Ft    781
Status         781
dtype: int64

In [216]:
data = data.drop('Price/SQ.Ft', axis = 1)

In [217]:
a= data['Location'].value_counts().keys()
data['Location'] = data['Location'].replace(to_replace=a[5:], value='Rare')
print data['Location'].value_counts()

Rare                  311
Santa Maria-Orcutt    265
Paso Robles            85
Atascadero             58
Nipomo                 34
Arroyo Grande          28
Name: Location, dtype: int64


In [218]:
dummies = pd.get_dummies(data['Location'])
for m_number in dummies.columns:
	data['Location_%s' % m_number] = dummies[m_number]

In [219]:
data = data.drop('Location', axis = 1)

In [220]:
print data.head()

      MLS     Price  Bedrooms  Bathrooms  Size      Status  \
0  132842  795000.0         3          3  2371  Short Sale   
1  134364  399000.0         4          3  2818  Short Sale   
2  135141  545000.0         4          3  3032  Short Sale   
3  135712  909000.0         4          4  3540  Short Sale   
4  136282  109900.0         3          1  1249  Short Sale   

   Location_Arroyo Grande  Location_Atascadero  Location_Nipomo  \
0                       1                    0                0   
1                       0                    0                0   
2                       0                    0                0   
3                       0                    0                0   
4                       0                    0                0   

   Location_Paso Robles  Location_Rare  Location_Santa Maria-Orcutt  
0                     0              0                            0  
1                     1              0                            0  
2             

In [221]:
data_new = data.drop(['MLS', 'Status'], axis = 1)

In [222]:
# for i in range(0,len(data.index)):
#     if data['Location'][i] not in a[0:7]:
#         data['Location'][i] = 'Rare'

In [223]:
type(data_new)

pandas.core.frame.DataFrame

In [224]:
from sklearn import preprocessing


In [225]:
scaler = preprocessing.MinMaxScaler()

In [226]:
test_data = scaler.fit_transform(data_new)

In [227]:
print test_data

[[ 0.14042942  0.3         0.2        ...,  0.          0.          0.        ]
 [ 0.06806761  0.4         0.2        ...,  1.          0.          0.        ]
 [ 0.09474646  0.4         0.2        ...,  1.          0.          0.        ]
 ..., 
 [ 0.06313385  0.3         0.1        ...,  0.          1.          0.        ]
 [ 0.10278666  0.3         0.1        ...,  0.          1.          0.        ]
 [ 0.19616263  0.3         0.2        ...,  0.          1.          0.        ]]


In [228]:
from sklearn.cluster import KMeans

In [229]:
data['Status'].value_counts()

Short Sale     516
Foreclosure    162
Regular        103
Name: Status, dtype: int64

In [237]:
clf = KMeans(n_clusters=3)

In [238]:
clf.fit(test_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [239]:
data['Predicted'] = clf.predict(test_data)

In [240]:
data['Predicted'].value_counts()

0    311
2    265
1    205
Name: Predicted, dtype: int64

In [241]:
data['Predicted'] = data['Predicted'].replace(to_replace=[0,1,2], value=['Short Sale', 'Foreclosure', 'REgular'])

In [242]:
data['Correct'] = data['Predicted']==data['Status']

In [243]:
data['Correct'].value_counts()

False    580
True     201
Name: Correct, dtype: int64